In [1]:
# Import necessary modules
import os
import mysql.connector as connector  # MySQL connector for database interaction
from mysql.connector import errorcode  # Import error codes for exception handling

# Load MySQL Credentials from Environment Variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST", "localhost")  # Default to localhost if not specified

# Establish MySQL Connection
try:
    connection = connector.connect(
        user=DB_USER,  
        password=DB_PASSWORD,  
        database=DB_NAME,
        host=DB_HOST
    )
    print("\nConnected to MySQL database")
except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Connection failed: Incorrect username or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(f"Error: {err}")
    exit()  # Exit script if connection fails

# Create cursor
cursor = connection.cursor()

# List All Databases
try:
    cursor.execute("SHOW DATABASES")
    databases = cursor.fetchall()
    print("\nList of Databases:")
    for db in databases:
        print(f"- {db[0]}")
except connector.Error as err:
    print(f"Error retrieving databases: {err}")

# List All Tables
try:
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    print("\nTables in LittleLemonDB:")
    for table in tables:
        print(f"- {table[0]}")
except connector.Error as err:
    print(f"Error retrieving tables: {err}")

# List All Stored Procedures
try:
    cursor.execute("""
        SELECT ROUTINE_NAME
        FROM INFORMATION_SCHEMA.ROUTINES
        WHERE ROUTINE_TYPE = 'PROCEDURE' AND ROUTINE_SCHEMA = %s
    """, (DB_NAME,))
    procedures = cursor.fetchall()
    
    print("\nStored Procedures in LittleLemonDB:")
    if procedures:
        for proc in procedures:
            print(f"- {proc[0]}")
    else:
        print("No stored procedures found.")
except connector.Error as err:
    print(f"Error retrieving stored procedures: {err}")

# List All Triggers
try:
    cursor.execute("""
        SELECT TRIGGER_NAME
        FROM INFORMATION_SCHEMA.TRIGGERS
        WHERE TRIGGER_SCHEMA = %s
    """, (DB_NAME,))
    triggers = cursor.fetchall()
    
    print("\nTriggers in LittleLemonDB:")
    if triggers:
        for trigger in triggers:
            print(f"- {trigger[0]}")
    else:
        print("No triggers found in LittleLemonDB.")
except connector.Error as err:
    print(f"Error retrieving triggers: {err}")

# Query to Retrieve Customer Details for Orders Over $40
try:
    query = """
        SELECT 
            cd.FirstName, 
            cd.LastName, 
            CONCAT(cd.PhoneNumber, ' - ', cd.Email) AS ContactDetails,
            o.TotalCost
        FROM Customer_Details AS cd
        JOIN Orders AS o ON cd.CustomerID = o.CustomerID
        WHERE o.TotalCost > 40;
    """
    cursor.execute(query)
    results = cursor.fetchall()

    print("\nCustomers Who Spent Over $40:")
    if results:
        for row in results:
            print(f"Name: {row[0]} {row[1]} | Contact: {row[2]} | Bill: ${row[3]}")
    else:
        print("No customers found who spent over $40.")
except connector.Error as err:
    print(f"Error retrieving customers: {err}")

# Close Connection
cursor.close()
connection.close()
print("\nMySQL connection closed.")


Connected to MySQL database

List of Databases:
- information_schema
- LittleLemonDB
- mysql
- performance_schema
- sys

Tables in LittleLemonDB:
- Audit_Log
- Bookings
- Customer_Details
- Menu
- Order_Delivery_Status
- Order_Items
- Orders
- Staff_Information
- Tables

Stored Procedures in LittleLemonDB:
- AddMultipleItemsToOrder
- AddValidBooking
- CancelAllBookingsForDate
- ChangeBookingTable
- CheckBooking
- ErrorHandler
- FindAvailableTables
- GetBookingsForDate
- GetMaxQuantity
- UpdateBooking
- UpdateOrderStatus

Triggers in LittleLemonDB:
- log_delete_booking
- log_insert_booking
- log_update_booking
- log_update_order_status
- update_order_total
- log_insert_order_items
- log_update_order_items
- log_delete_order_items
- log_insert_order
- log_new_order_status
- after_order_cancelled

Customers Who Spent Over $40:
Name: Jane Doe | Contact: 555-0002 - jane.doe@example.com | Bill: $100.00
Name: Charlie Brown | Contact: 555-0005 - charlie.brown@example.com | Bill: $83.50
Name: 